# **Machine-to-Machine (M2M) Landsat-9 Search and Download**

#### **What is M2M?**
> EarthExplorer (EE) serves as a key data access portal, offering a range of tools for searching, discovering, and downloading data and metadata from the USGS Earth Resources Observation and Science (EROS) data repository. The Machine-to-Machine (M2M) API allows users to search and retrieve download URLS and metadata from the EROS archive by using programming languages like Python or PHP. Users can create JSON structures to pass to M2M endpoints, subsequently receiving JSON responses in return.

#### **Requesting Access**
> Users will need to log on to their [**EROS Registration Service (ERS)**](https://ers.cr.usgs.gov/) accounts to view the M2M documentation. To submit download requests through M2M, ERS users must be authorized. 
> Steps to request M2M access:
> 1. Login to [**EROS Registration Service (ERS)**](https://ers.cr.usgs.gov/) (See [**How to Create an ERS Account**](https://www.youtube.com/watch?v=Ut6kxbuP_nk))
> 2. Select the **Access Request** menu
> 3. From the **Access Controls** page, use the **Request Access** button
> 4. In the **Access Type** selector, choose **Access to EE's Machine to Machine interface (MACHINE)**, then complete the questions


<div class="alert alert-warning"><h4>
The USGS/EROS User Services team assesses these requests. Contact USGS EROS User Services for more information: <a href="https://www.usgs.gov/staff-profiles/usgs-eros-customer-services" target="_blank">custserv@usgs.gov</a> and visit the <a href="https://www.usgs.gov/centers/eros/science/earthexplorer-help-index#ers" target="_blank">EarthExplorer Help Index</a>   </h4></div>


#### **Use Case Scenario**

> This tutorial is designed to guide users through metadata filtering for Landsat-9 satellite data within the Landsat 8-9 Operational Land Imager and Thermal Infrared Sensor Collection 2 Level-2 dataset. The chosen location for used is Albemarle Sound, a large estuarine body of water located in northeastern North Carolina, United States. The emphasis on metadata filtering is crucial, especially when dealing with large Landsat datasets. The Landsat archives house vast amounts of data, and applying metadata filters becomes essential to streamline the selection process. Thus this notebook focuses on filtering the available metadata to download Landsat-9 products within a user-define area of interest (AOI).
>
> This example shows the user how to:
> 
> 1. [**Create an Area of Interest GeoJSON file**](#createaoi)
> 2. [**Use *dataset-search* to find Landsat 8-9 Scenes**](#dataset-search)
> 3. [**Search for queryable metadata with a *dataset-filter***](#dataset-filters)
> 5. [**Submit a *scene-search* using *scene-filters***](#scene-search)
> 6. [**Find the *download-options* for the scenes**](#download-options)
> 7. [**Request a direct download URL for each product using *download-request***](#download-request)
> 8. [**Submit a *download-retrieve* request**](#download-retrieve)
> 9. [***Logout* of M2M API**](#logout-api)

<div class="alert alert-success" style="font-size: 14px;">
    <h4>Queryable Metadata Parameters</h4>
    <p>The set of queryable metadata parameters may vary depending on the dataset collection. For instance, Landsat 8-9 OLI/TIRS, Landsat 7 ETM+, and Landsat 4-5 TM may not share the same set of parameters available for querying.</p>
    <p>Users can explore queryable metadata through one of the following methods:</p>
    <ul>
        <li>Submit a <a href="https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters">dataset-filters</a> query and review the results.</li>
        <li>Perform a search on <a href="https://earthexplorer.usgs.gov/">Earth Explorer</a> and access the list of metadata available for querying under the <i><b>Additional Information</b></i> tab.</li>
    </ul>
    <p>Refer to the <a href="https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary">Landsat Collection 2 Data Dictionary</a> for a comprehensive description of all queryable metadata available through Earth Explorer.</p>
</div>


## **Setup**

- ### Import necessary libraries

In [1]:
import json
from getpass import getpass
import requests
import sys
import time
import cgi
import os
import pandas as pd
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")

- ### Define send request function

    This function sends a request to a M2M endpoint and returns the parsed JSON response.

    Input parameters include:
    - **endpoint_url** (*str*): The URL of the M2M endpoint
    - **payload** (*dict*): The payload to be sent with the request

    Returns:
    - **dict**: Parsed JSON response

In [2]:
# Send http request
def sendRequest(url, data, apiKey = None, exitIfNoResponse = True):
    """
    Send a request to an M2M endpoint and returns the parsed JSON response.

    Parameters:
    endpoint_url (str): The URL of the M2M endpoint
    payload (dict): The payload to be sent with the request

    Returns:
    dict: Parsed JSON response
    """
      
    json_data = json.dumps(data)
    
    if apiKey == None:
        response = requests.post(url, json_data)
    else:
        headers = {'X-Auth-Token': apiKey}              
        response = requests.post(url, json_data, headers = headers)  
    
    try:
      httpStatusCode = response.status_code 
      if response == None:
          print("No output from service")
          if exitIfNoResponse: sys.exit()
          else: return False
      output = json.loads(response.text)
      if output['errorCode'] != None:
          print(output['errorCode'], "- ", output['errorMessage'])
          if exitIfNoResponse: sys.exit()
          else: return False
      if  httpStatusCode == 404:
          print("404 Not Found")
          if exitIfNoResponse: sys.exit()
          else: return False
      elif httpStatusCode == 401: 
          print("401 Unauthorized")
          if exitIfNoResponse: sys.exit()
          else: return False
      elif httpStatusCode == 400:
          print("Error Code", httpStatusCode)
          if exitIfNoResponse: sys.exit()
          else: return False
    except Exception as e: 
          response.close()
          print(e)
          if exitIfNoResponse: sys.exit()
          else: return False
    response.close()
    
    return output['data']

- ### Define download function

In [3]:
def downloadfiles(downloadIds):
    downloadIds.append(download['downloadId'])
    print("    DOWNLOADING: " + download['url'])
    downloadResponse = requests.get(download['url'], stream=True)

    # parse the filename from the Content-Disposition header
    content_disposition = cgi.parse_header(downloadResponse.headers['Content-Disposition'])[1]
    filename = os.path.basename(content_disposition['filename'])
    filepath = os.path.join(data_dir, filename)

    # write the file to the destination directory
    with open(filepath, 'wb') as f:
        for data in downloadResponse.iter_content(chunk_size=8192):
            f.write(data)
    #print(f"    DOWNLOADED {filename} ({i+1}/{len(downloadIds)})")

- ### Set up`data` directory

In [4]:
data_dir = 'data'
utils_dir = 'utils'
dirs = [ data_dir, utils_dir]

for d in dirs:
        if not os.path.exists(d): 
            try: 
                os.makedirs(d)
                print(f"Directory '{d}' created successfully.") 
            except OSError as e: 
                print(f"Error creating directory '{d}': {e}") 
        else: 
            print(f"Directory '{d}' already exists.") 


Directory 'data' created successfully.
Directory 'utils' already exists.


---
## **ERS User Login**

<div class="alert alert-warning">
    <h3>The USGS M2M Server will no longer support using your ERS <b>passwords</b> for authentication in scripts but instead requires <b>tokens</b>. For more information, visit the <a href="https://www.usgs.gov/media/files/m2m-application-token-documentation" target="_blank">M2M Application Token Documentation</a> or contact <a href="mailto:custserv@usgs.gov" target="_blank">USGS EROS User Services</a>.</h3>
    <h4><p>To get your token, please follow these steps:</p>
    <ol>
        <li>Navigate to <a href="https://ers.cr.usgs.gov/" target="_blank">ERS</a> and log in to your ERS account.</li>
        <li>Click "Create Application Token" under the "Application Tokens" section of the profile page.</li>
        <li>Set the "Token Name" and "Token Expiration."</li>
        <li>Make sure to copy/save your token so it is easily accessible and can be used to download data with the API.</li>
    </ol>
    </h4>
</div>

- ### Login by entering [EROS Registration System (EROS)](https://ers.cr.usgs.gov/https://ers.cr.usgs.gov/) ***`username`*** and ***`token`*** when prompted

In [5]:
def prompt_ERS_login(serviceURL):
    print("Logging in...\n")

    p = ['Enter EROS Registration System (ERS) Username: ', 'Enter ERS Account Token: ']

    # Use requests.post() to make the login request
    response = requests.post(f"{serviceUrl}login-token", json={'username': getpass(prompt=p[0]), 'token': getpass(prompt=p[1])})

    if response.status_code == 200:  # Check for successful response
        apiKey = response.json()['data']
        print('\nLogin Successful, API Key Received!')
        headers = {'X-Auth-Token': apiKey}
        return apiKey
    else:
        print("\nLogin was unsuccessful, please try again or create an account at: https://ers.cr.usgs.gov/register.")

In [6]:
serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"
apiKey = prompt_ERS_login(serviceUrl)

# Print the API key only if login was successful
# if apiKey:
#     print("\nAPI Key: " + apiKey + "\n"))


Logging in...



Enter EROS Registration System (ERS) Username:  ········
Enter ERS Account Token:  ········



Login Successful, API Key Received!


#### **OR**

- ### Login by setting [EROS Registration System (EROS)](https://ers.cr.usgs.gov/https://ers.cr.usgs.gov/) ***`username`*** and ***`token`***

In [7]:
# username = ""
# token = ""

In [8]:
# print("Logging in...\n")
    
# serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"
# login_payload = {'username' : username, 'token' : token}
    
# apiKey = sendRequest(serviceUrl + "login-token", login_payload)
    
# print("API Key: " + apiKey + "\n")

## **1. Create an Area of Interest (AOI) GeoJSON Text File** <a id="createaoi"></a>

Polygon coordinates are arranged in the sequence NE, NW, SW, SE, NE, representing all four corners, with an extra NE point to conclude and close the polygon.

In [9]:
from geojson import Polygon, Feature, FeatureCollection, dump

polygon = Polygon([[[-75.4460, 36.3793],\
                  [-76.9196, 36.3793],\
                  [-76.9196, 35.5713],\
                  [-75.4460, 35.5713],\
                  [-75.4460, 36.3793]]])

features = []
features.append(Feature(geometry=polygon, properties={"state":"North Carolina"}))

# add more features...
# features.append(...)

feature_collection = FeatureCollection(features)

with open('./utils/north_carolina_aoi.geojson', 'w') as f:
    dump(feature_collection, f)

- ### Open the GeoJSON file as a geopandas dataframe

In [10]:
aoi_geodf =  gpd.read_file('./utils/north_carolina_aoi.geojson') #aoi geopandas dataframe

- ### View the Coordinate Reference System (CRS)

> The CRS used is the World Geodetic System 1984 with units in degrees.

In [11]:
aoi_geodf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

- ### Plot AOI using Folium

> The ***folium*** module is used for creating interactive web maps.

In [12]:
import folium
m = folium.Map(location=[aoi_geodf.centroid.y[0], aoi_geodf.centroid.x[0]], zoom_start=8, tiles="openstreetmap",\
              width="90%",height="90%",attributionControl=0) #add n estimate of where the center of the polygon would be located\
                                        #for the location [latitude longitude]

In [13]:
for _, r in aoi_geodf.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "blue"})
    geo_j.add_to(m)
m

## **2. Submit a [*dataset-search*](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-search)**<a id="dataset-search"></a>

The [***dataset-search***](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-search) is used to find available collections of datasets. Users can pass the API without search parameters to view all the available datasets; however, in this case, we use the ***'datasetName'*** input parameter to limit the search to find datasets with ***'Landsat 8-9'*** since Landsat-8 and Landsat-9 datasets are grouped.  

- ### Setup search  parameters into a ***payload*** used for sending requests:

> Adding a [***temporal-filter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#temporalFilter) will return data collections available within a set period of time. The ***'start'*** and ***'end'*** dates in ISO format (**yyyy-mm-dd**) for the time period of interest.

In [14]:
temporalFilter = {'start' : '2023-03-01', 'end' : '2023-04-01'}

- ### Include ***'Landsat 8-9'*** as ***'datasetName'***:

> Note that ***'Landsat 8-9'***  is not a ***'datasetName'*** but can be used to query for specifically Landsat-8 or Landsat-9 data.

In [15]:
datasearch_payload = {'datasetName': 'Landsat 8-9',
                     'temporalFilter' : temporalFilter}

In [16]:
datasearch_result = sendRequest(serviceUrl + "dataset-search", datasearch_payload, apiKey)

<div class="alert alert-info">
    <h4>To find the <b><i>datasetName</i></b> for other collections of Landsat data:</h4>
    <ol>
        <li>
            Run a request with the <a href="https://m2m.cr.usgs.gov/api/docs/reference/#dataset-search" target="_blank">
                <b><i>dataset-search</i></b>
            </a> request and extract the <b><i>'datasetAlias'</i></b> field. No input parameters are necessary!
        </li>
        <li>
            Run the <b><i>dataset-search</i></b> endpoint using the
            <a href="https://m2m.cr.usgs.gov/api/test/json/" target="_blank">
                <b><i>Machine-to-Machine (M2M) Test Page</i></b>.
            </a>
        </li>
    </ol>
</div>


- ### Visualize results using *pandas*:

In [17]:
pd.json_normalize(datasearch_result)

,abstractText,acquisitionStart,acquisitionEnd,catalogs,collectionName,collectionLongName,datasetId,datasetAlias,datasetCategoryName,dataOwner,...,legacyId,sceneCount,temporalCoverage,supportCloudCover,supportDeletionSearch,allowInKmz,spatialBounds.north,spatialBounds.east,spatialBounds.south,spatialBounds.west
0,The USGS Earth Resources Observation and Scien...,2013-04-11,None,"[EE, GV]",Landsat 8-9 OLI/TIRS C2 L1,Landsat 8-9 Operational Land Imager and Therma...,5e81f14f59432a27,landsat_ot_c2_l1,Landsat Collection 2 Level-1,LSAA,...,9825,3683317,"[""2013-03-08 00:00:00-06"",""2024-08-21 00:00:00...",True,True,True,84.441827,197.576528,-84.571377,-180.009967
1,The USGS Earth Resources Observation and Scien...,2013-04-11,None,[EE],Landsat 8-9 OLI/TIRS C2 L2,Landsat 8-9 Operational Land Imager and Therma...,5e83d14f2fc39685,landsat_ot_c2_l2,Landsat Collection 2 Level-2,LSAA,...,9885,3194012,"[""2013-03-18 00:00:00-05"",""2024-08-20 00:00:00...",True,True,True,84.441827,194.444152,-84.517917,-180.009967


## **3. Send a [*dataset-filters*](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) Request** <a id="dataset-filters"></a>
The [***dataset-filters***](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) request returns the [***metadataFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter) parameters for the specified dataset. These values can be used as additional criteria when submitting search queries for specific scenes. The ***'datasetName'*** input parameter is necessary.  

> Note that the Landsat-9 Collection 2 Level 2 datasets has the ***dataAlias:*** ***'landsat_ot_c2_l2'***

In [18]:
datasetName = datasearch_result[1]['datasetAlias']

In [19]:
datafilter_payload = {'datasetName': datasetName}
datafilter_result = sendRequest(serviceUrl + "dataset-filters", datafilter_payload, apiKey)

> Below we plot the results from the [***dataset-filters***](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) query. The **`fieldLabel`** provides insights into the queryable fields available for this specific Landsat data collection. While the **`fieldConfig.type`** that lists types `Text`, `Range` and `Select` guides users on the appropriate [***metadataFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter) type to apply.
    
> **Example:**
> - If type is `Text`, then use a `'like'` or `'='` operand in a `'value'` metadataFilter
> - If type is `Select`, then check the `valueList` for valid values to use in a `'value'` metadataFilter (use the key on the left, not the label on the right)
> - If type is `Range`, then use a `'between'` metadataFilter and provide `'firstValue'` and `'secondValue'`

In [20]:
pd.json_normalize(datafilter_result)

,id,legacyFieldId,dictionaryLink,fieldLabel,searchSql,fieldConfig.type,fieldConfig.filters,fieldConfig.options.size,fieldConfig.validators,fieldConfig.numElements,...,valueList.8,fieldConfig.options.multiple,valueList.OLI,valueList.OLI_TIRS,valueList.TIRS,valueList.NADIR,valueList.OFFNADIR,valueList.T1,valueList.T2,valueList.RT
0,5e83d14f567d0086,27819.0,https://www.usgs.gov/centers/eros/science/land...,Landsat Product Identifier L2,EE_DISPLAY_ID like ?,Text,"[{'type': 'StringToUpper', 'options': []}, {'t...",50,[],5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5e83d14f227b0d85,27863.0,https://www.usgs.gov/centers/eros/science/land...,Landsat Product Identifier L1,LANDSAT_PRODUCT_ID_L1 like ?,Text,"[{'type': 'StringToUpper', 'options': []}, {'t...",45,[],5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5e83d14fc84c9a78,27836.0,https://www.usgs.gov/centers/eros/science/land...,Landsat Scene Identifier,ENTITY_ID like ?,Text,"[{'type': 'StringToUpper', 'options': []}, {'t...",30,[],5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5e83d14fb9436d88,27820.0,https://www.usgs.gov/centers/eros/science/land...,WRS Path,WRS_PATH between ?,Range,[],NaN,[],1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5e83d14ff1eda1b8,27821.0,https://www.usgs.gov/centers/eros/science/land...,WRS Row,WRS_ROW between ?,Range,[],NaN,[],1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,61af9273566bb9a8,NaN,https://www.usgs.gov/centers/eros/science/land...,Satellite,satellite = ?,Select,[],NaN,[],NaN,...,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5e83d14fc6e09eb6,27827.0,https://www.usgs.gov/centers/eros/science/land...,Sensor Identifier,SENSOR_ID = ?,Select,"[{'type': 'StringToUpper', 'options': []}]",4,[],1,...,NaN,True,OLI,OLI_TIRS,TIRS,NaN,NaN,NaN,NaN,NaN
7,5f6aa429b870fd7f,NaN,https://www.usgs.gov/centers/eros/science/land...,Nadir/Off Nadir,NADIR_OFFNADIR = ?,Select,"[{'type': 'StringToUpper', 'options': {}}]",NaN,[],NaN,...,NaN,NaN,NaN,NaN,NaN,Nadir,Off Nadir,NaN,NaN,NaN
8,5f6a6fb2137a3c00,NaN,https://www.usgs.gov/centers/eros/science/land...,Collection Category,COLLECTION_CATEGORY = ?,Select,[],NaN,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tier 1,Tier 2,Real-Time
9,5f6a6f82795f1b6c,NaN,https://www.usgs.gov/centers/eros/science/land...,Date Product Generated L2 (YYYY/MM/DD),trunc(DATE_L2_GENERATED) between ?,Range,[],NaN,"[{'type': 'Date', 'options': {}}]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## **4. Setup *scene-filters* and to submit a *scene-search*:**<a id="scene-search"></a>

- ### Setup [***Scene-filters***](https://m2m.cr.usgs.gov/api/docs/datatypes/#sceneFilter):

> The [***spatialFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#spatialFilter), [***acquisitionFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#acquisitionFilter), [**metadataFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter) and [***cloudCoverFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#cloudCoverFilter) are [***scene-filters***](https://m2m.cr.usgs.gov/api/docs/datatypes/#sceneFilter) used below to identify the scenes matching the filter criteria using [**scene-search**](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search).
>
> #### [***spatialFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#spatialFilter)
>
> There are two spatial ***filterType's***:
>
> 1. [***geojson***](https://m2m.cr.usgs.gov/api/docs/datatypes/#spatialFilterGeoJson) which needs a ***['geoJson'](https://m2m.cr.usgs.gov/api/docs/datatypes/#geoJson)*** field name labeled with a geometric ***'type'*** e.g. ***'Polygon'*** and includes the coordinate array of the polygon:
>
> ~~~~
>        spatialFilter = {'filterType' : 'geojson',
>                 'geoJson' : {'type': 'Polygon',\ 
>                              'coordinates': [[[-75.446, 36.3793],\
>                                                                  [-76.9196, 36.3793],\
>                                                                  [-76.9196, 35.5713],
>                                                                  [-75.446, 35.5713],\
>                                                                  [-75.446, 36.3793]]]}}
>
> ~~~~
>
> 2. [***mbr***](https://m2m.cr.usgs.gov/api/docs/datatypes/#spatialFilterMbr) (Minimum Bounding Rectangle) where the user should include the ***'lowerLeft'*** (southwest point) and ***'upperRight'*** (northeast point) of the rectangle as ***{'latitude': ######, 'longitude' : ######}*** coordinates:
> 
> ```
>        spatialFilter =  {'filterType' : 'mbr',
>                      'lowerLeft' : {'latitude' : 35.5713,\
>                                     'longitude' : -76.9196},
>                     'upperRight' : { 'latitude' : 36.3793,\
>                                     'longitude' : -75.446}}
> ```

In [21]:
spatialFilter =  {'filterType' : 'mbr',
                    'lowerLeft' : {'latitude' : aoi_geodf.bounds.miny[0],\
                                   'longitude' : aoi_geodf.bounds.minx[0]},
                   'upperRight' : { 'latitude' : aoi_geodf.bounds.maxy[0],\
                                   'longitude' : aoi_geodf.bounds.maxx[0]}}

# spatialFilter = {'filterType' : 'geojson',
#                  'geoJson' : {'type': 'Polygon',\ 
#                               'coordinates': [[[-75.446, 36.3793],\
#                                                                   [-76.9196, 36.3793],\
#                                                                   [-76.9196, 35.5713],
#                                                                   [-75.446, 35.5713],\
#                                                                   [-75.446, 36.3793]]]}}

> #### [***acquisitionFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#acquisitionFilter) 
> The [***acquisitionFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#acquisitionFilter) is the ***'start'*** and ***'end'*** dates in ISO format for the time period of interest. In our case it will be the same as the ***temporalFilter*** used above. It is also in ISO format **yyyy-mm-dd**. 

In [22]:
temporalFilter = {'start' : '2023-03-01', 'end' : '2023-04-01'}

> #### [***metadataFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter)
> The [***metadataFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter) is used to narrow down the scene search to find specific scenes e.g. in our case searching only for Landsat-9 images. A ***'filterId'*** (from the [***dataset-filters***](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) result), ***'filterType'*** and ***'value'*** (9 for Landsat-9) are necessary when searching for datasets from a specific satellite.

In [23]:
filterId = datafilter_result[5]['id'] #because the id for the satellite metadata filter
#filterId = datasearch_result[1]['datasetId']#gives error: Expecting ',' delimiter: line 1 column 163 (char 162)
metadataFilter = {'filterType': 'value',
                    'filterId': filterId,
                    'value' : '9'}

> #### [***cloudCoverFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#cloudCoverFilter) 
> To filter scenes by cloud cover, you can use the [***cloudCoverFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#cloudCoverFilter) and specify a ***'min'*** and ***'max'*** percentage (%) value..

In [24]:
cloudCoverFilter = {'min' : 0, 'max' : 20}

- ### Combine all parameters into a *payload* used for sending requests:

In [25]:
search_payload = {
    'datasetName' : datasetName,
    'sceneFilter' : {
        'metadataFilter' : metadataFilter,
        'spatialFilter' : spatialFilter,
        'acquisitionFilter' : temporalFilter,
        'cloudCoverFilter' : cloudCoverFilter
    }
}

In [26]:
search_payload

{'datasetName': 'landsat_ot_c2_l2',
 'sceneFilter': {'metadataFilter': {'filterType': 'value',
   'filterId': '61af9273566bb9a8',
   'value': '9'},
  'spatialFilter': {'filterType': 'mbr',
   'lowerLeft': {'latitude': 35.5713, 'longitude': -76.9196},
   'upperRight': {'latitude': 36.3793, 'longitude': -75.446}},
  'acquisitionFilter': {'start': '2023-03-01', 'end': '2023-04-01'},
  'cloudCoverFilter': {'min': 0, 'max': 20}}}

- ### Submit a [***scene-search***](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search)


> The [***scene-search***](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) is used for searching for scenes within a dataset collection, a ***'datasetName'*** parameter is required.

In [27]:
scenes = sendRequest(serviceUrl + "scene-search", search_payload, apiKey)

In [28]:
pd.json_normalize(scenes['results'])

,browse,cloudCover,entityId,displayId,orderingId,metadata,hasCustomizedMetadata,publishDate,options.bulk,options.download,...,options.secondary,selected.bulk,selected.compare,selected.order,spatialBounds.type,spatialBounds.coordinates,spatialCoverage.type,spatialCoverage.coordinates,temporalCoverage.endDate,temporalCoverage.startDate
0,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",13,LC90130352023085LGN00,LC09_L2SP_013035_20230326_20230328_02_T1,None,[],None,2023-03-28 00:03:28-05,True,True,...,False,False,False,False,Polygon,"[[[-75.92757, 34.97982], [-75.92757, 37.09251]...",Polygon,"[[[-75.92757, 35.37131], [-73.88723, 34.97982]...",2023-03-26 00:00:00,2023-03-26 00:00:00
1,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",14,LC90150352023083LGN00,LC09_L2SP_015035_20230324_20230326_02_T1,None,[],None,2023-03-25 23:28:12-05,True,True,...,False,False,False,False,Polygon,"[[[-79.02454, 34.97983], [-79.02454, 37.0925],...",Polygon,"[[[-79.02454, 35.37128], [-76.98435, 34.97983]...",2023-03-24 00:00:00,2023-03-24 00:00:00
2,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",0,LC90150352023067LGN00,LC09_L2SP_015035_20230308_20230310_02_T1,None,[],None,2023-03-09 23:23:47-06,True,True,...,False,False,False,False,Polygon,"[[[-79.01536, 34.97965], [-79.01536, 37.0924],...",Polygon,"[[[-79.01536, 35.37118], [-76.97556, 34.97965]...",2023-03-08 00:00:00,2023-03-08 00:00:00


- ### Create a list of ***SceneIds***

> The ***entityIds*** are [Landsat Scene Identifiers](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_scene_id) that remain consistent for all levels and products available for each scene. Below we group all entityIds into a variable called *`sceneIds`*.

In [29]:
sceneIds = []
for result in scenes['results']:
    # Add this scene to the list I would like to download
    sceneIds.append(result['entityId'])
    
sceneIds

['LC90130352023085LGN00', 'LC90150352023083LGN00', 'LC90150352023067LGN00']

## **5. Find the [***download-options***](https://m2m.cr.usgs.gov/api/docs/reference/#download-options) for the scenes**<a id="download-options"></a>

The [***download-options***](https://m2m.cr.usgs.gov/api/docs/reference/#download-options) request is used to discover downloadable products for each dataset. If a download is marked as not available, an order must be placed to generate that product. A ***'datasetName'*** parameter is required.

In [30]:
download_payload = {'datasetName' : datasetName, 
                    'entityIds' : sceneIds}

downloadOptions = sendRequest(serviceUrl + "download-options", download_payload, apiKey)


In [31]:
pd.json_normalize(downloadOptions)

,id,downloadName,displayId,entityId,datasetId,available,filesize,productName,productCode,bulkAvailable,downloadSystem,secondaryDownloads,fileGroups
0,5e83d14fec7cae84,None,LC09_L2SP_013035_20230326_20230328_02_T1,LC90130352023085LGN00,5e83d14f2fc39685,True,818830718,Landsat Collection 2 Level-2 Product Bundle,D694,True,ls_zip,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
1,6448198cc7b442a4,C2L2 Tile Product Files,LC09_L2SP_013035_20230326_20230328_02_T1,LC90130352023085LGN00,5e83d14f2fc39685,True,0,Landsat Collection 2 Level-2 Band File,D693,True,folder,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
2,6448198c62023764,C2L2 Tile Product Files,LC09_L2SP_013035_20230326_20230328_02_T1,LC90130352023085LGN00,5e83d14f2fc39685,True,0,Landsat Collection 2 Level-2 Band File,D691,True,folder,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
3,632210d4770592cf,None,LC09_L2SP_013035_20230326_20230328_02_T1,LC90130352023085LGN00,5e83d14f2fc39685,False,818830718,Landsat Collection 2 Level-2 Product Bundle,D806,False,dds_ms,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
4,5e83d14fec7cae84,None,LC09_L2SP_015035_20230308_20230310_02_T1,LC90150352023067LGN00,5e83d14f2fc39685,True,945831610,Landsat Collection 2 Level-2 Product Bundle,D694,True,ls_zip,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
5,6448198cc7b442a4,C2L2 Tile Product Files,LC09_L2SP_015035_20230308_20230310_02_T1,LC90150352023067LGN00,5e83d14f2fc39685,True,0,Landsat Collection 2 Level-2 Band File,D693,True,folder,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
6,6448198c62023764,C2L2 Tile Product Files,LC09_L2SP_015035_20230308_20230310_02_T1,LC90150352023067LGN00,5e83d14f2fc39685,True,0,Landsat Collection 2 Level-2 Band File,D691,True,folder,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
7,632210d4770592cf,None,LC09_L2SP_015035_20230308_20230310_02_T1,LC90150352023067LGN00,5e83d14f2fc39685,False,945831610,Landsat Collection 2 Level-2 Product Bundle,D806,False,dds_ms,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
8,5e83d14fec7cae84,None,LC09_L2SP_015035_20230324_20230326_02_T1,LC90150352023083LGN00,5e83d14f2fc39685,True,958605073,Landsat Collection 2 Level-2 Product Bundle,D694,True,ls_zip,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
9,6448198cc7b442a4,C2L2 Tile Product Files,LC09_L2SP_015035_20230324_20230326_02_T1,LC90150352023083LGN00,5e83d14f2fc39685,True,0,Landsat Collection 2 Level-2 Band File,D693,True,folder,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None


<div class="alert alert-block alert-warning">
    <h4> <b>NOTE:</b> The scene list cannot exceed 50,000 items. </h4>
</div>  

- ### Create a list of available products using results from [***download-options***](https://m2m.cr.usgs.gov/api/docs/reference/#download-options)

> Make sure the product is available for this scene and ignore `'folders'` since they are tile files:

In [32]:
availableproducts = []
for product in downloadOptions:
        # Make sure the product is available for this scene
        if product['available'] == True and product['downloadSystem'] != 'folder':
                availableproducts.append({'entityId' : product['entityId'],
                                   'productId' : product['id']})

In [33]:
availableproducts

[{'entityId': 'LC90130352023085LGN00', 'productId': '5e83d14fec7cae84'},
 {'entityId': 'LC90150352023067LGN00', 'productId': '5e83d14fec7cae84'},
 {'entityId': 'LC90150352023083LGN00', 'productId': '5e83d14fec7cae84'}]

## **6. Submit a [*download-request*](https://m2m.cr.usgs.gov/api/docs/reference/#download-request)**<a id="download-request"></a>

The [**download-request**](https://m2m.cr.usgs.gov/api/docs/reference/#download-request) is used to request download urls and adds them to a queue.

In [34]:
requestedDownloadsCount = len(availableproducts)

# set a label for the download request
label = "download-sample"
download_req_payload = {'downloads' : availableproducts,
                                 'label' : label}

requestResults = sendRequest(serviceUrl + "download-request", download_req_payload, apiKey)
requestResults

{'availableDownloads': [{'downloadId': 659719300,
   'eulaCode': None,
   'url': 'https://landsatlook.usgs.gov/gen-bundle?landsat_product_id=LC09_L2SP_013035_20230326_20230328_02_T1&requestSignature=eyJkb3dubG9hZEFwcCI6Ik0yTSIsImNvbnRhY3RJZCI6MjcxMDAwMzEsImRvd25sb2FkSWQiOjY1OTcxOTMwMCwiZGF0ZUdlbmVyYXRlZCI6IjIwMjQtMDgtMjJUMTA6MjQ6NDUtMDU6MDAiLCJpZCI6IkxDMDlfTDJTUF8wMTMwMzVfMjAyMzAzMjZfMjAyMzAzMjhfMDJfVDEiLCJzaWduYXR1cmUiOiIkNSQkNUlTcTVvUEZGNE5DVnNSN1VYOUxJTnRCMzFMREFWcFdmU0JtZ2FCT2wuXC8ifQ=='},
  {'downloadId': 659719301,
   'eulaCode': None,
   'url': 'https://landsatlook.usgs.gov/gen-bundle?landsat_product_id=LC09_L2SP_015035_20230308_20230310_02_T1&requestSignature=eyJkb3dubG9hZEFwcCI6Ik0yTSIsImNvbnRhY3RJZCI6MjcxMDAwMzEsImRvd25sb2FkSWQiOjY1OTcxOTMwMSwiZGF0ZUdlbmVyYXRlZCI6IjIwMjQtMDgtMjJUMTA6MjQ6NDUtMDU6MDAiLCJpZCI6IkxDMDlfTDJTUF8wMTUwMzVfMjAyMzAzMDhfMjAyMzAzMTBfMDJfVDEiLCJzaWduYXR1cmUiOiIkNSQkbVM2aWx3REFFXC9LM2tSbVBvRkJFRDdsQml2T1dOaDZFbzg2NFYyeFZ3SzAifQ=='},
  {'downloadId': 6597193

## **7. Submit a [*download-retrieve*](https://m2m.cr.usgs.gov/api/docs/reference/#download-retrieve)**<a id="download-retrieve"></a>

Here we call the [***download-retrieve***](https://m2m.cr.usgs.gov/api/docs/reference/#download-retrieve) method to get products that are not immediately available for download, these are listed under the **`'preparingDownloads'`** results after sending a ***download-request***. The download links that are immediately available are retrieved from the **`'availableDownloads'`** list.

In [35]:
 if requestResults['preparingDownloads'] != None and len(requestResults['preparingDownloads']) > 0:
    download_retrieve_payload = {'label' : label}
    
    print("Requesting for additional available download urls...")
    moreDownloadUrls = sendRequest(serviceUrl + "download-retrieve", download_retrieve_payload, apiKey)

    downloadIds = []  



    print("\nDownloading from available downloads:")    
    for download in moreDownloadUrls['available']:
        if str(download['downloadId']) in requestResults['newRecords'] or str(download['downloadId']) in requestResults['duplicateProducts']:
            downloadfiles(downloadIds)




    print("\nDownloading from requested downloads:")        
    for download in moreDownloadUrls['requested']:
        if str(download['downloadId']) in requestResults['newRecords'] or str(download['downloadId']) in requestResults['duplicateProducts']:
            downloadfiles(downloadIds)               



    # Didn't get all of the reuested downloads, call the download-retrieve method again probably after 30 seconds
    while len(downloadIds) < (requestedDownloadsCount - len(requestResults['failed'])): 
        preparingDownloads = requestedDownloadsCount - len(downloadIds) - len(requestResults['failed'])
        print("    ", preparingDownloads, " downloads are not available. Waiting for 30 seconds...")
        time.sleep(30)
        print("    Trying to retrieve data after waiting for 30 seconds...")
        moreDownloadUrls = sendRequest(serviceUrl + "download-retrieve", download_retrieve_payload, apiKey)
        for download in moreDownloadUrls['available']:                            
            if download['downloadId'] not in downloadIds and (str(download['downloadId']) in requestResults['newRecords'] or str(download['downloadId']) in requestResults['duplicateProducts']):
                downloadfiles(downloadIds)


else:
    print("\nAll downloads are available to download. Retrieving...\n")# Get all available downloads
    i = 0
    for download in requestResults['availableDownloads']:
        
        print("DOWNLOADING: " + download['url'])
        
        downloadResponse = requests.get(download['url'], stream=True)

        # parse the filename from the Content-Disposition header
        content_disposition = cgi.parse_header(downloadResponse.headers['Content-Disposition'])[1]
        filename = os.path.basename(content_disposition['filename'])
        filepath = os.path.join(data_dir, filename)

        # write the file to the destination directory
        with open(filepath, 'wb') as f:
            for data in downloadResponse.iter_content(chunk_size=8192):
                f.write(data)
        i+=1
        print(f"DOWNLOADED {filename} ({i}/{len(requestResults['availableDownloads'])})\n")
        



All downloads are available to download. Retrieving...

DOWNLOADING: https://landsatlook.usgs.gov/gen-bundle?landsat_product_id=LC09_L2SP_013035_20230326_20230328_02_T1&requestSignature=eyJkb3dubG9hZEFwcCI6Ik0yTSIsImNvbnRhY3RJZCI6MjcxMDAwMzEsImRvd25sb2FkSWQiOjY1OTcxOTMwMCwiZGF0ZUdlbmVyYXRlZCI6IjIwMjQtMDgtMjJUMTA6MjQ6NDUtMDU6MDAiLCJpZCI6IkxDMDlfTDJTUF8wMTMwMzVfMjAyMzAzMjZfMjAyMzAzMjhfMDJfVDEiLCJzaWduYXR1cmUiOiIkNSQkNUlTcTVvUEZGNE5DVnNSN1VYOUxJTnRCMzFMREFWcFdmU0JtZ2FCT2wuXC8ifQ==
DOWNLOADED LC09_L2SP_013035_20230326_20230328_02_T1.tar (1/3)

DOWNLOADING: https://landsatlook.usgs.gov/gen-bundle?landsat_product_id=LC09_L2SP_015035_20230308_20230310_02_T1&requestSignature=eyJkb3dubG9hZEFwcCI6Ik0yTSIsImNvbnRhY3RJZCI6MjcxMDAwMzEsImRvd25sb2FkSWQiOjY1OTcxOTMwMSwiZGF0ZUdlbmVyYXRlZCI6IjIwMjQtMDgtMjJUMTA6MjQ6NDUtMDU6MDAiLCJpZCI6IkxDMDlfTDJTUF8wMTUwMzVfMjAyMzAzMDhfMjAyMzAzMTBfMDJfVDEiLCJzaWduYXR1cmUiOiIkNSQkbVM2aWx3REFFXC9LM2tSbVBvRkJFRDdsQml2T1dOaDZFbzg2NFYyeFZ3SzAifQ==
DOWNLOADED LC09_L2SP_01503

## **8. [*Logout*](https://m2m.cr.usgs.gov/api/docs/reference/#logout) of M2M Endpoint**<a id="logout-api"></a>

Logout so the API Key cannot be used anymore.

<div class="alert alert-block alert-warning">
    <h4> <b>NOTE:</b> The Machine-to-Machine API key expires after 2 hours of inactivity. </h4>
</div>  

In [36]:
endpoint = "logout"  
if sendRequest(serviceUrl + endpoint, None, apiKey) == None:        
    print("\n\nLogged Out\n")
else:
    print("\n\nLogout Failed\n")



Logged Out



## **9. List Downloads**

In [37]:
os.listdir(data_dir)

['LC09_L2SP_015035_20230308_20230310_02_T1.tar',
 'LC09_L2SP_013035_20230326_20230328_02_T1.tar',
 'LC09_L2SP_015035_20230324_20230326_02_T1.tar']


<div class="alert alert-block alert-info">
    <h1> Contact Information </h1>
    <h3> Material written by Tonian Robinson<sup>1</sup> </h3>
    <ul>
        <b>Contact:</b> custserv@usgs.gov <br> 
        <b>Voice:</b> +1-605-594-6151 <br>
        <b>Organization:</b> USGS EROS User Services <br>
        <b>Date last modified:</b> 22-Aug-2024 <br>
    </ul>
    
<sup>1</sup>Earth Space Technology Services LLC., contractor to the U.S. Geological Survey, Earth Resources Observation and Science (EROS) Center, Sioux Falls, South Dakota, 57198-001, USA. Work performed under USGS contract G0121D0001.
</div>